# Importing Libraries and Defining Functions

In [3]:
from pptx import Presentation
import pandas as pd
import os

def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

def retrieve_effort(file_name):
    prs = Presentation(file_name)
    result = pd.DataFrame()

    for slide in prs.slides:
        for shape in slide.shapes:
            if not shape.has_table:
                continue
            tbl = shape.table
            row_count = len(tbl.rows)
            col_count = len(tbl.columns)

            if (
                ("teams" in tbl.cell(0, 0).text.lower() or "team" in tbl.cell(0, 0).text.lower())
                and tbl.cell(0, 0).text.lower() != ""
                and tbl.cell(0, 2).text.lower() not in "support"
            ):
                for r in range(1, row_count):
                    col1 = []
                    col3 = []

                    for c in range(0, col_count):
                        text_runs = []
                        cell = tbl.cell(r, c)
                        paragraphs = cell.text_frame.paragraphs

                        for paragraph in paragraphs:
                            for run in paragraph.runs:
                                before = run.text.split()
                                store = ""

                                if c != 2:
                                    for i in before:
                                        i = i.replace("-", " ")
                                        i = i.replace("–", " ")
                                        i = i.strip()

                                        if len(i) > 0:
                                            store = store + i[0].upper() + i[1:] + " "
                                        else:
                                            store = store + i + " "
                                        if len(store.strip()) == 0:
                                            store = "No value"
                                else:
                                    for i in before:
                                        i = i.strip()

                                        if len(i) > 0:
                                            store = store + i[0] + i[1:] + " "
                                        else:
                                            store = store + i + " "
                                        if len(store.strip()) == 0:
                                            store = "No value"

                                if c == 3 and "effort" in tbl.cell(0, 3).text.lower():
                                    store = ""
                                    for i in before:
                                        i = i.strip()

                                        if len(i) > 0:
                                            store = store + i[0] + i[1:] + " "
                                        else:
                                            store = store + i + " "
                                        if len(store.strip()) == 0:
                                            store = "No value"

                                text_runs.append(store.strip())

                        if c == 0:
                            if len(text_runs) > 0 and text_runs[0] != "Program Manager":
                                col1.append("".join(text_runs))
                            else:
                                col1.append("".join(text_runs))
                        elif c == 3 and "effort" in tbl.cell(0, 3).text.lower():
                            col3.append("".join(text_runs))
                        elif c == 2:
                            col3.append("".join(text_runs))

                    if "effort" in tbl.cell(0, 3).text.lower():
                        val = col3[-1]
                        col3 = []
                        col3.append(val)

                    data_frame = pd.DataFrame(list(zip(col1, col3)), columns=["Teams", "Effort"])
                    result = pd.concat([result, data_frame])

                effort_col = tbl.cell(0, 2).text.lower()

    if len(result) > 0:
        not_clean_teams = result["Teams"].tolist()
        not_clean_effort = result["Effort"].tolist()
        teams = []

        for team_item in not_clean_teams:
            teams.append((team_item.encode("ascii", errors="backslashreplace").decode()).replace("\\u200b", ""))

        effort = []

        for item in not_clean_effort:
            if item.lower() in ("", "n/a", "na"):
                effort.append("")
            else:
                item = (item.encode("ascii", errors="backslashreplace").decode()).replace("\\u200b", "")

                if (item[-2:].lower() in ("mm")) and (isfloat(item[:-2])):
                    new_item = item[0:-2].split(" ")

                    if isfloat(new_item[0]):
                        effort.append(float(new_item[0]))
                    else:
                        effort.append(item)

                elif ("mm" in effort_col.lower()) and (item.isnumeric()):
                    effort.append(float(item))

                elif ("sm" in effort_col.lower()) and (item.isnumeric()):
                    effort.append(item + " SM")

                elif item.lower() in (
                    "via rserp process",
                    "refer to windows topsheet",
                    "absorb",
                    "as needed",
                    "refer to windows topsheet",
                    "minimal",
                    "absorbed",
                    "no added effort*",
                    "covered by ml plan",
                    "completed",
                    "sustaining",
                    "no incremental effort",
                    "?",
                    "debug support",
                    "no added effort *",
                    "na",
                    "as needed for debug",
                ):
                    effort.append(item)

                else:
                    new_item = item
                    effort.append(new_item)

        result = [list(x) for x in zip(teams, effort)]

    return result


# Extracting Effort Data from PowerPoint Files


In [4]:
dictSWRP = {}

for folders in os.listdir("."):
    if folders == "SWRP":
        for swrp_folders in os.listdir(folders):
            if swrp_folders != ".DS_Store":
                list_of_files = filter(
                    lambda x: os.path.isfile(os.path.join("./" + folders + "/" + swrp_folders, x)),
                    os.listdir("./" + folders + "/" + swrp_folders),
                )
                list_of_files = sorted(
                    list_of_files, key=lambda x: os.path.getmtime(os.path.join("./" + folders + "/" + swrp_folders, x))
                )

                for top_sheet in list_of_files:
                    if "~$" not in top_sheet and top_sheet.endswith(".pptx"):
                        dictSWRP[swrp_folders] = retrieve_effort("./" + folders + "/" + swrp_folders + "/" + top_sheet)

effort_data = []
teams_data = []

for swrp in dictSWRP:
    for item in dictSWRP[swrp]:
        teams_data.append(item[0])
        effort_data.append(item[1])


# Retrieving Excel Data


In [5]:
def retrieve_excel(excel_file):
    data = pd.read_excel(r"{0}".format(excel_file))
    excel_data_frame_new = pd.DataFrame(data)
    excel_data_frame_new.drop([0, 1, excel_data_frame_new.shape[0] - 1], axis=0, inplace=True)
    excel_data_frame_new = excel_data_frame_new.reset_index(drop=True)
    new_header = excel_data_frame_new.iloc[0]
    excel_data_frame_new = excel_data_frame_new[1:]
    excel_data_frame_new.columns = new_header
    excel_data_frame_new.reset_index(drop=True)
    excel_data_frame_new = excel_data_frame_new[
        ["Key", "Summary", "Program Manager", "Roadmap Release - Linux", "Roadmap Release - Windows", "Status"]
    ]

    return excel_data_frame_new


# Combining Effort Data with Excel Data


In [6]:
excel_data_frame = pd.DataFrame(
    columns=["Key", "Summary", "Program Manager", "Roadmap Release - Linux", "Roadmap Release - Windows", "Status"]
)

for folders in os.listdir("."):
    if folders == "Excel":
        for excel_files in os.listdir(folders):
            if ("~$" not in excel_files) and (excel_files.endswith(".xlsx")):
                excel_data_frame = pd.concat([excel_data_frame, retrieve_excel("./Excel/" + excel_files)])

excel_data_frame.drop_duplicates(inplace=True)
excel_data_frame.set_index("Key", inplace=True)
teams_data = []

for swrp_folders in dictSWRP:
    for item in dictSWRP[swrp_folders]:
        if (item[0] != "Program Manager") and (len(item[0]) != 0):
            teams_data.append(item[0])

teams_data = list(set(teams_data))
unique_teams = list(set(teams_data))
teams_data.sort()

for cols in teams_data:
    excel_data_frame[str(cols)] = None

excel_data_frame.sort_index()


/var/folders/9p/w309j4n94633ymfb_v3x990r0000gn/T/ipykernel_19000/3692525476.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  excel_data_frame[str(cols)] = None
/var/folders/9p/w309j4n94633ymfb_v3x990r0000gn/T/ipykernel_19000/3692525476.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  excel_data_frame[str(cols)] = None
/var/folders/9p/w309j4n94633ymfb_v3x990r0000gn/T/ipykernel_19000/3692525476.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

,Summary,Program Manager,Roadmap Release - Linux,Roadmap Release - Windows,Status,2D,3D,3D CQE,3rd Party Vendor For Translation,AMF,...,Virt(Cloud),VirtIQE,Virtualization,Virtualization CQE,Virtualization IQE,Virtualization Linux,Virtualization Windows,Vulkan,Vulkan/OGL,Web Team
Key,,,,,,,,,,,,,,,,,,,,,
SWRP-1053,Chrome OS for Cezanne/Barcelo (Monkey Island),"Tabian, Reza",NaN,22.1,Approved,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
SWRP-1099,Streaming SDK 2.0,"Samuelson, Shawn",22.1,NaN,Sustaining,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
SWRP-1114,dGPU Power Optimization w/ FreeSync- MultiDisplay,"Pillai, Rajitha",NaN,22.1,Rejected,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
SWRP-1143,HIP & MathLibs for ISV Apps (Linux),"Wang, Angela",22.1,NaN,Approved,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
SWRP-1164,OpenGL on PAL (Cloud)- Linux,"Ma, Jason(Jun)",22.2,NaN,Approved,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWRP-1415,Windows Feb 2023 CI,"Chiu, Steven",NaN,22.4,Approved,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
SWRP-1448,Intake temperature sensor,"Madhunapantula, Sekhar",NaN,22.4,Approved,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
SWRP-453,Radeon Raytracing Analyzer 1.0,"Samuelson, Shawn",NaN,22.2,Sustaining,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


# Populating Excel Data with Effort Information


In [7]:
for release in dictSWRP:
    teams_data = []
    effort_data = []

    for item in range(len(dictSWRP[release])):
        teams_data.append(dictSWRP[release][item][0])
        effort_data.append(dictSWRP[release][item][1])

    count = 0

    for cols in teams_data:
        if cols in excel_data_frame.columns:
            excel_data_frame.loc[str(release), cols] = effort_data[count]
            count += 1

excel_data_frame.insert(0, "SWRP", excel_data_frame.index.tolist())
excel_data_frame = excel_data_frame.reset_index(drop=True)
excel_data_frame["SWRP"] = excel_data_frame["SWRP"].str[5:]
excel_data_frame["SWRP"] = excel_data_frame["SWRP"].astype(int)

excel_data_frame.to_excel("PPTExcel696.xlsx", index=False)


/var/folders/9p/w309j4n94633ymfb_v3x990r0000gn/T/ipykernel_19000/1474011514.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  excel_data_frame.insert(0, "SWRP", excel_data_frame.index.tolist())
